In [1]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import math
import glob

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

import sklearn 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.utils import class_weight


import HigherTierModel

In [2]:
###########################################################
# Define file
###########################################################

trainVarFile = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/higherTier_Y.npz'

In [3]:
###########################################################
# Hyperparameters
###########################################################

nVariables = 12
         
ntrain = 221
ntest  = 24

nEpochs = 10
batchSize = 64
learningRate = 1e-4

In [4]:
###########################################################
# Get data from file
###########################################################

data = np.load(trainVarFile)
    
variables_train = data['variables_train']
variables_test = data['variables_test']
    
y_train = data['y_train']
y_test = data['y_test']

print('variables_train.shape:', variables_train.shape)
print('variables_test.shape:', variables_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

variables_train.shape: (221, 12)
variables_test.shape: (24, 12)
y_train.shape: (221, 1)
y_test.shape: (24, 1)


In [5]:
###########################################################
# Define model
###########################################################

model = HigherTierModel.HigherTierModel(nVariables)

In [6]:
###########################################################
# Define the optimiser and compile the model
###########################################################

optimiser = optimizers.legacy.Adam(learning_rate=learningRate)
model.compile(loss='binary_crossentropy', optimizer=optimiser, metrics=['accuracy'])

In [7]:
###########################################################
# Fit the model
###########################################################

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6, verbose=1)

history = model.fit(variables_train, y_train, validation_data=(variables_test, y_test), batch_size=batchSize, shuffle=True, epochs=nEpochs) 

Epoch 1/10
4/4 [==============================] - 0s 40ms/step - loss: 0.2726 - accuracy: 0.9864 - val_loss: 8.9646e-05 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 0s 3ms/step - loss: 0.2335 - accuracy: 0.9864 - val_loss: 1.7942e-04 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 0s 3ms/step - loss: 0.2043 - accuracy: 0.9864 - val_loss: 3.7316e-04 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 0s 3ms/step - loss: 0.1584 - accuracy: 0.9864 - val_loss: 7.9117e-04 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 3ms/step - loss: 0.1275 - accuracy: 0.9864 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 3ms/step - loss: 0.1176 - accuracy: 0.9819 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 3ms/step - loss: 0.0925 - accuracy: 0.9819 - val_loss: 0.0050 - val_accuracy: 1.0000
Epoch 8/10
4/4 [=